In [13]:
from tkinter import *
from tkinter import colorchooser
from PIL import ImageTk, Image
%run ./base_exercises.ipynb
%run ./exercise_data.ipynb

# Global variable to keep track of the current exercise index
# When moving to the next/prev exercise, this index changes accordingly
current_index = 0

# Dictionary to store text colors for each exercise bc each might have different colors 
text_colors = {}

# Dictionary to store current colors for different parts of the exercise details
current_colors = {
    'name_label': 'black',
    'name': 'black',
    'description_label': 'black',
    'description': 'black',
    'duration_label': 'black',
    'duration': 'black',
    'difficulty_label': 'black',
    'difficulty': 'black',
    'muscles_label': 'black',
    'muscles': 'black',
    'instructions_label': 'black',
    'instructions': 'black',
    'textcolor': 'red'
}

# Create a Tkinter window
window = Tk()
window.title("Exercise Details")
icon = PhotoImage(file='../Shared Components/Assets/workout_icon.png')
window.iconphoto(True, icon)
# window.config(bg="cornflowerblue")

# Function to display exercise details in the window
def display_exercise_details():
    # Access global var to get current index
    global current_index
    
    # Get the current exercise to retrieve the details to be displayed
    exercise = exercises[current_index]
    
    # Initialize color dictionary for current exercise if not already done
    if exercise.name not in text_colors:
        text_colors[exercise.name] = current_colors.copy() # Creates a new dict w/ default colors for the current exercise
    
    # Clears previous labels & texts
    for widget in window.winfo_children():
        widget.destroy()
        
    # Function to choose color
    def choose_color(label_key, label):
        color_code = colorchooser.askcolor(title="Choose Text Color")[1]
        if color_code:
            label.config(fg=color_code)
            text_colors[exercise.name][label_key] = color_code
            # Update current colors to reflect the new choice for subsequent exercises
            current_colors[label_key] = color_code

    # Create labels to display exercise details
    def create_label(text, row, column, font=("Verdana", 15, 'bold'), label_key=None, colspan=1):
        # Determining the text color to use for this label
        fg_color = text_colors[exercise.name].get(label_key, current_colors[label_key])
        
        # Creates the label w/ the specified text, font, & color
        label = Label(window, text=text, font=font, fg=fg_color)
        
        # Places the label in the grid layout at the specified row/col
        label.grid(row=row, column=column, columnspan=colspan, sticky="w")
        
        # If label_key provided, makes the label clickable to change its color
        if label_key:
            label.bind("<Button-1>", lambda e: choose_color(label_key, label))
            
        # Returns the created label
        return label
        
    # Name Label
    create_label("Name:", 1, 0, label_key='name_label')
    create_label(exercise.name, 1, 1, font=("Verdana", 15, 'bold'), label_key='name')

    # Description Label
    create_label("Description:", 2, 0, label_key='description_label')
    create_label(exercise.description, 2, 1, label_key='description')
    
    # Duration Label
    create_label("Duration:", 3, 0, label_key='duration_label')
    create_label(str(exercise.duration) + " seconds", 3, 1, label_key='duration')
    
    # Difficulty Label
    create_label("Difficulty:", 4, 0, label_key='difficulty_label')
    create_label(exercise.difficulty, 4, 1, label_key='difficulty')
    
    # Muscle Groups Label
    create_label("Muscles Worked:", 5, 0, label_key='muscles_label')
    muscle_groups_str = ", ".join(exercise.muscle_groups)
    create_label(muscle_groups_str, 5, 1, label_key='muscles')
    
    # Instructions Label
    instructions_label = create_label("Instructions:", 6, 0, label_key='instructions_label')
    instructions_text = Text(window, height=6, width=40, wrap=WORD, fg=text_colors[exercise.name]['instructions'])
    instructions_text.insert(END, exercise.instructions)
    instructions_text.config(state=DISABLED, font=("Verdana", 15, 'bold'))  # Make the text entry field read-only
    instructions_text.grid(row=6, column=1, sticky="w")
    instructions_text.bind("<Button-1>", lambda e: choose_color('instructions', instructions_text))
    
    # Click on text Label
    create_label("Click on any text to change its color :D", 7, 1, font=("Verdana", 13), label_key='textcolor', colspan=1)
    

    # Load/Displaying the Workout Images
    img = Image.open(exercise.image_path)
    img = img.resize((300, 300), Image.LANCZOS)
    photo_img = ImageTk.PhotoImage(img)  # Converts PIL image object into PhotoImage 
    img_label = Label(window, image=photo_img)
    img_label.image = photo_img  # Keeps a reference to avoid garbage collection
    img_label.grid(row=1, column=2, rowspan=6)
    
    # Creating Navigation Buttons
    prev_btn = Button(window, text="Previous", command=show_prev_exercise)
    prev_btn.grid(row=7, column=0, sticky="w")
    
    next_btn = Button(window, text="Next", command=show_next_exercise)
    next_btn.grid(row=7, column=1, sticky="e")
    
# Function to show previous exercise
def show_prev_exercise():
    global current_index
    current_index = (current_index - 1) % len(exercises)
    display_exercise_details()

# Function to show next exercise
def show_next_exercise():
    global current_index
    current_index = (current_index + 1) % len(exercises)
    display_exercise_details()    

# Displaying the first exercise
display_exercise_details()

# Configure grid row and column weights to make them expandable
window.grid_rowconfigure(0, weight=1)
window.grid_rowconfigure(1, weight=1)
window.grid_columnconfigure(1, weight=1)
window.grid_columnconfigure(2, weight=1)

window.mainloop()
